In [118]:
import cv2
from matplotlib import pyplot as plt
import os
import torch
import numpy as np
from queue import PriorityQueue
import time
import psutil

In [119]:
def measure_processing_power(func, *args, **kwargs):
    """
    Measure CPU and memory usage during the execution of a function.

    Args:
        func (callable): The function to measure.
        *args: Positional arguments for the function.
        **kwargs: Keyword arguments for the function.

    Returns:
        result: The result of the function execution.
        stats: A dictionary containing CPU and memory usage stats.
    """
    # Record initial stats
    process = psutil.Process()
    start_cpu = process.cpu_percent(interval=None)
    start_mem = process.memory_info().rss
    start_time = time.perf_counter()

    # Run the function
    result = func(*args, **kwargs)

    # Record final stats
    end_cpu = process.cpu_percent(interval=None)
    end_mem = process.memory_info().rss
    end_time = time.perf_counter()

    # Calculate deltas
    cpu_usage = end_cpu - start_cpu
    mem_usage = end_mem - start_mem
    duration = end_time - start_time

    stats = {
        "CPU_Usage (%)": cpu_usage,
        "Memory_Usage (bytes)": mem_usage,
        "Duration (seconds)": duration
    }

    return result, stats

In [120]:
def draw_detections(image, detections, label_map=None, color=(0, 255, 0), thickness=5):
    """
    Draw bounding boxes and labels on an image to visualize detections.

    Args:
        image (ndarray): The image on which to draw detections.
        detections (list): List of detections. Each detection is a dictionary containing:
            - 'x_min': Minimum x-coordinate of the bounding box.
            - 'y_min': Minimum y-coordinate of the bounding box.
            - 'x_max': Maximum x-coordinate of the bounding box.
            - 'y_max': Maximum y-coordinate of the bounding box.
            - 'label' (optional): Label or class ID of the detection.
        label_map (dict, optional): A mapping from class IDs to human-readable labels.
        color (tuple): Color of the bounding box (default: green).
        thickness (int): Thickness of the bounding box lines (default: 2).

    Returns:
        annotated_image (ndarray): The image with drawn detections.
    """
    annotated_image = image.copy()

    for detection in detections:
        x_min = int(detection['x_min'])
        y_min = int(detection['y_min'])
        x_max = int(detection['x_max'])
        y_max = int(detection['y_max'])
        label = detection.get('label', None)

        # Draw the bounding box
        cv2.rectangle(annotated_image, (x_min, y_min), (x_max, y_max), color, thickness)

        # Draw the label if available
        if label is not None:
            label_text = str(label_map[label]) if label_map and label in label_map else str(label)
            cv2.putText(
                annotated_image,
                label_text,
                (x_min, y_min - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                2,
                color,
                5
            )

    return annotated_image

def save_yolo_labels( labels, output_dir, image_name ):
    os.makedirs(f"{output_dir}/labels", exist_ok=True)

    label_path = f"{output_dir}/labels/{image_name}.txt"
    with open(label_path, "w") as f:
        for label in labels:
            f.write(" ".join(map(str, label)) + "\n")
    print(f"Saved YOLO label file: {label_path}")

def save_annotated_image( image, detections, output_dir, image_name, suffix="" ):
    os.makedirs(f"{output_dir}/images", exist_ok=True)
    annotated_image = draw_detections(image, detections)
    annotated_image_path = f"{output_dir}/images/{image_name}_result{suffix}.jpg"
    cv2.imwrite(annotated_image_path, annotated_image)
    print(f"Saved annotated image: {annotated_image_path}")

In [121]:
def qr_code_detector(image,):
    """
    Detect QR codes using OpenCV and save results in YOLO format.

    Args:
        image_path (str): Path to the input image.
        output_dir (str): Directory to save results (images and labels).
        save_annotated (bool): If True, save the annotated image with QR code detections.
    """
    
    # Load the image
    h, w = image.shape[:2]
    
    # Initialize QR Code detector
    qr_detector = cv2.QRCodeDetector()

    # Detect and decode QR codes
    data, _, vertices, _ = qr_detector.detectAndDecodeMulti(image)

    detections = []  # Store detection information for drawing
    labels = []      # Store YOLO-format labels

    if vertices is not None:
        for i, points in enumerate(vertices):
            # Calculate bounding box coordinates
            x_min = points[:, 0].min()
            y_min = points[:, 1].min()
            x_max = points[:, 0].max()
            y_max = points[:, 1].max()

            # Append to detections for visualization
            detections.append({
                'x_min': x_min,
                'y_min': y_min,
                'x_max': x_max,
                'y_max': y_max,
                'label': f"QR {i}"  # Optional label for visualization
            })

            # Calculate YOLO-format labels
            x_center = (x_min + x_max) / 2 / w  # Normalize by image width
            y_center = (y_min + y_max) / 2 / h  # Normalize by image height
            width = (x_max - x_min) / w         # Normalize width
            height = (y_max - y_min) / h        # Normalize height

            labels.append([0, x_center, y_center, width, height])  # Assuming class_id = 0 for QR codes
    else:
        print(f"No QR codes detected in the image")

    return detections, labels

In [ ]:
# Example Usage
test_dir = "../data/test/samples/"
cv2_results_dir = "../data/test/results/cv2_detector/"

# Run QR code detection and save results for each image in the test directory
for filename in os.listdir(test_dir):
    if filename.endswith((".jpg", ".png", ".jpeg", ".JPG")) and "out" not in filename and filename[0] != ".":
        filepath = os.path.join(test_dir, filename)
        image_name = os.path.splitext(os.path.basename(filepath))[0]
        print(f"Processing: {filepath}")
        image = cv2.imread(filepath)
        if image is None:
            raise ValueError(f"Error: Could not load image at {filepath}")
        cv2_detections, cv2_labels = qr_code_detector(image)
        print(cv2_detections)
        save_yolo_labels(cv2_labels, cv2_results_dir, image_name)
        save_annotated_image( image, cv2_detections, cv2_results_dir, image_name)

Processing: ../data/test/samples/test_light_130.JPG
[{'x_min': 1908.8589, 'y_min': 3035.5122, 'x_max': 1994.0, 'y_max': 3157.0, 'label': 'QR 0'}, {'x_min': 1903.0, 'y_min': 2585.0, 'x_max': 1992.0, 'y_max': 2708.0, 'label': 'QR 1'}]
Saved YOLO label file: ../data/test/results/cv2_detector//labels/test_light_130.txt
Saved annotated image: ../data/test/results/cv2_detector//images/test_light_130_result.jpg
Processing: ../data/test/samples/test_light_120.JPG
[{'x_min': 1776.0, 'y_min': 2511.7073, 'x_max': 1873.1279, 'y_max': 2644.0, 'label': 'QR 0'}, {'x_min': 2317.0, 'y_min': 3028.6, 'x_max': 2440.049, 'y_max': 3183.338, 'label': 'QR 1'}]
Saved YOLO label file: ../data/test/results/cv2_detector//labels/test_light_120.txt
Saved annotated image: ../data/test/results/cv2_detector//images/test_light_120_result.jpg
Processing: ../data/test/samples/test_110.JPG
[{'x_min': 2557.0, 'y_min': 2240.0, 'x_max': 2761.0, 'y_max': 2484.0, 'label': 'QR 0'}, {'x_min': 1717.1102, 'y_min': 3058.7502, 'x_ma

In [123]:
def yolov5n_detector(image, model):
    """
    Perform inference with YOLOv5 and save the results as annotated images and YOLO labels.

    Args:
        model: Loaded YOLOv5 model.
        image_path (str): Path to the input image.
        output_dir (str): Directory to save results (images and labels).
    """

    
    # Load the image
    h, w = image.shape[:2]
    
    # Perform inference
    results = model(image)

    # Process detections
    detections = []
    labels = []
    for _, row in results.pandas().xyxy[0].iterrows():
        x_min = row['xmin']
        y_min = row['ymin']
        x_max = row['xmax']
        y_max = row['ymax']
        class_id = int(row['class'])

        detections.append({
            'x_min': x_min,
            'y_min': y_min,
            'x_max': x_max,
            'y_max': y_max,
            'label': class_id
        })

        # Normalize for YOLO format
        x_center = (x_min + x_max) / 2 / w
        y_center = (y_min + y_max) / 2 / h
        width = (x_max - x_min) / w
        height = (y_max - y_min) / h

        labels.append([class_id, x_center, y_center, width, height])

    return detections, labels

In [124]:



# Load the model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../models/yolov5n_best.pt')

# Verify the model is loaded
print("Model loaded successfully!")

# Input and output paths
test_dir = "../data/test/samples/"
yolo_results_dir = "../data/test/results/yolov5n/"

# Run QR code detection and save results for each image in the test directory
for filename in os.listdir(test_dir):
    if filename.endswith((".jpg", ".png", ".jpeg", ".JPG")) and "out" not in filename and filename[0] != ".":
        filepath = os.path.join(test_dir, filename)
        image_name = os.path.splitext(os.path.basename(filepath))[0]
        print(f"Processing: {filepath}")
        image = cv2.imread(filepath)
        if image is None:
            raise ValueError(f"Error: Could not load image at {filepath}")
        yolo_detections, yolo_labels = yolov5n_detector(image, model)
        print(yolo_detections)
        save_yolo_labels(yolo_labels, yolo_results_dir, image_name)
        save_annotated_image( image, cv2_detections, cv2_results_dir, image_name)

Using cache found in /Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-12 Python-3.11.11 torch-2.5.1 CPU

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 
/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Model loaded successfully!
Processing: ../data/test/samples/test_light_130.JPG
[{'x_min': 2417.516845703125, 'y_min': 2572.69189453125, 'x_max': 2528.23388671875, 'y_max': 2691.4248046875, 'label': 0}, {'x_min': 2428.328857421875, 'y_min': 3100.721435546875, 'x_max': 2520.1484375, 'y_max': 3200.66796875, 'label': 0}, {'x_min': 1914.5042724609375, 'y_min': 3091.47900390625, 'x_max': 1985.30615234375, 'y_max': 3157.51513671875, 'label': 0}, {'x_min': 1895.886474609375, 'y_min': 2585.45458984375, 'x_max': 1973.0841064453125, 'y_max': 2673.521484375, 'label': 0}]
Saved YOLO label file: ../data/test/results/yolov5n//labels/test_light_130.txt
Saved annotated image: ../data/test/results/cv2_detector//images/test_light_130_result.jpg
Processing: ../data/test/samples/test_light_120.JPG
[{'x_min': 2333.33984375, 'y_min': 3054.1123046875, 'x_max': 2438.814697265625, 'y_max': 3178.10498046875, 'label': 0}, {'x_min': 2335.583984375, 'y_min': 2499.048828125, 'x_max': 2452.3134765625, 'y_max': 2626.7

/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved annotated image: ../data/test/results/cv2_detector//images/test_light_120_result.jpg
Processing: ../data/test/samples/test_110.JPG
[{'x_min': 2564.634765625, 'y_min': 2241.718994140625, 'x_max': 2769.63037109375, 'y_max': 2467.222900390625, 'label': 0}, {'x_min': 1712.2957763671875, 'y_min': 2307.879638671875, 'x_max': 1864.3236083984375, 'y_max': 2499.845703125, 'label': 0}, {'x_min': 2563.293701171875, 'y_min': 3120.527587890625, 'x_max': 2752.515625, 'y_max': 3317.4228515625, 'label': 0}, {'x_min': 1718.5660400390625, 'y_min': 3074.98779296875, 'x_max': 1855.0997314453125, 'y_max': 3262.615966796875, 'label': 0}]
Saved YOLO label file: ../data/test/results/yolov5n//labels/test_110.txt


/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


[{'x_min': 1678.4371337890625, 'y_min': 2502.092041015625, 'x_max': 1778.25341796875, 'y_max': 2622.346923828125, 'label': 0}, {'x_min': 2289.668212890625, 'y_min': 3117.958740234375, 'x_max': 2402.458740234375, 'y_max': 3236.90966796875, 'label': 0}, {'x_min': 2281.39306640625, 'y_min': 2479.793212890625, 'x_max': 2406.967529296875, 'y_max': 2623.405517578125, 'label': 0}, {'x_min': 1687.268310546875, 'y_min': 3087.055908203125, 'x_max': 1758.6217041015625, 'y_max': 3171.270751953125, 'label': 0}]
Saved YOLO label file: ../data/test/results/yolov5n//labels/test_light_110.txt
Saved annotated image: ../data/test/results/cv2_detector//images/test_light_110_result.jpg
Processing: ../data/test/samples/test_140.JPG


/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


[{'x_min': 1658.0960693359375, 'y_min': 3112.94580078125, 'x_max': 1775.2559814453125, 'y_max': 3255.72021484375, 'label': 0}, {'x_min': 2324.142578125, 'y_min': 2469.575439453125, 'x_max': 2474.67578125, 'y_max': 2639.33447265625, 'label': 0}, {'x_min': 1651.5953369140625, 'y_min': 2501.152587890625, 'x_max': 1771.2982177734375, 'y_max': 2631.46826171875, 'label': 0}, {'x_min': 4121.7509765625, 'y_min': 3149.30810546875, 'x_max': 4202.1142578125, 'y_max': 3221.99365234375, 'label': 0}]
Saved YOLO label file: ../data/test/results/yolov5n//labels/test_140.txt
Saved annotated image: ../data/test/results/cv2_detector//images/test_140_result.jpg
Processing: ../data/test/samples/test_light_100.JPG
[{'x_min': 2324.877685546875, 'y_min': 2490.340087890625, 'x_max': 2479.45166015625, 'y_max': 2659.4775390625, 'label': 0}, {'x_min': 1683.723876953125, 'y_min': 2501.897705078125, 'x_max': 1787.30322265625, 'y_max': 2609.648193359375, 'label': 0}, {'x_min': 1677.6610107421875, 'y_min': 3114.44995

/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [130]:
def format_detections_for_transform(detections, image_shape):
    """
    Reorders detections into srcPoints corresponding to the quadrants 
    (top-left, top-right, bottom-left, bottom-right) for homography calculation.

    Args:
        detections (list): List of dictionaries with bounding box details.
        image_shape (tuple): Shape of the image as (height, width).

    Returns:
        list: List of four (x, y) source points in the correct order.
    """
    h, w = image_shape[:2]

    # Initialize lists for each quadrant
    top_left, top_right, bottom_left, bottom_right = [], [], [], []

    tl_point = (-1,-1)
    br_point = (-1,-1)

    for detection in detections:
        x_min = detection['x_min']
        y_min = detection['y_min']
        x_max = detection['x_max']
        y_max = detection['y_max']
        center_x = (x_min + x_max) / 2
        center_y = (y_min + y_max) / 2

        # Assign to quadrants
        if center_x < w / 2 and center_y < h / 2:
            top_left.append((center_x, center_y))
            tl_point = (x_min, y_min)
        elif center_x >= w / 2 and center_y < h / 2:
            top_right.append((center_x, center_y))
        elif center_x < w / 2 and center_y >= h / 2:
            bottom_left.append((center_x, center_y))
        else:
            bottom_right.append((center_x, center_y))
            br_point = (x_max, y_max)

    # Verify we have one point per quadrant
    if len(top_left) != 1 or len(top_right) != 1 or len(bottom_left) != 1 or len(bottom_right) != 1:
        raise ValueError("Four QR codes (one in each quadrant) are required for the transform.")

    # Return points in the order: top-left, top-right, bottom-left, bottom-right
    out_points = [top_left[0], top_right[0], bottom_right[0], bottom_left[0]]
    return out_points, tl_point, br_point

In [131]:
filepath = "../data/targets/output/scoring_target_qr_5_h.jpg"
# Load the model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../models/yolov5n_best.pt')

image_name = os.path.splitext(os.path.basename(filepath))[0]
print(f"Processing: {filepath}")
image = cv2.imread(filepath)
if image is None:
    raise ValueError(f"Error: Could not load image at {filepath}")


result, stats = measure_processing_power(qr_code_detector, image)
cv2_detections, cv2_labels = result
print(cv2_detections)
print(stats)
save_annotated_image( image, cv2_detections, "../data/targets/sandbox_run/cv2/", image_name)
result, stats = measure_processing_power(yolov5n_detector, image, model)
yolo_detections, yolo_labels = result
print(yolo_detections)
print(stats)
save_annotated_image( image, yolo_detections, "../data/targets/sandbox_run/yolo/", image_name)

dst_points, _, _ = format_detections_for_transform(cv2_detections, image.shape)

Using cache found in /Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-12 Python-3.11.11 torch-2.5.1 CPU

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 


Processing: ../data/targets/output/scoring_target_qr_5_h.jpg
[{'x_min': 70.0, 'y_min': 70.0, 'x_max': 579.0, 'y_max': 579.0, 'label': 'QR 0'}, {'x_min': 2713.0, 'y_min': 63.0, 'x_max': 3236.0, 'y_max': 586.0, 'label': 'QR 1'}, {'x_min': 77.89289, 'y_min': 1978.0, 'x_max': 571.0, 'y_max': 2473.1882, 'label': 'QR 2'}, {'x_min': 2719.0, 'y_min': 1970.0, 'x_max': 3229.0, 'y_max': 2479.0, 'label': 'QR 3'}]
{'CPU_Usage (%)': 215.9, 'Memory_Usage (bytes)': -16646144, 'Duration (seconds)': 0.7085000420047436}
Saved annotated image: ../data/targets/scanned/cv2//images/scoring_target_qr_5_h_result.jpg
[]
{'CPU_Usage (%)': 466.2, 'Memory_Usage (bytes)': 43974656, 'Duration (seconds)': 0.06255654199048877}
Saved annotated image: ../data/targets/scanned/yolo//images/scoring_target_qr_5_h_result.jpg


/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [132]:
def find_four_point_transform(srcPoints, dstPoints):
    """Solves for and returns a perspective transform.

    Each source and corresponding destination point must be at the
    same index in the lists.

    Do not use the following functions (you will implement this yourself):
        cv2.findHomography
        cv2.getPerspectiveTransform
    Hint: You will probably need to use least squares to solve this.
    Args:
        srcPoints (list): List of four (x,y) source points
        dstPoints (list): List of four (x,y) destination points
    Returns:
        numpy.array: 3 by 3 homography matrix of floating point values
    """

    # Generate pair of equations for each point
    p = srcPoints
    t = dstPoints
    
    A = np.zeros((8, 9))
    
    for i in range(len(p)):
        A[2 * i + 0] = [p[i][0], p[i][1], 1, 0, 0, 0, -t[i][0] * p[i][0], -t[i][0] * p[i][1], -t[i][0]]
        A[2 * i + 1] = [0, 0, 0, p[i][0], p[i][1], 1,  -t[i][1] * p[i][0], -t[i][1] * p[i][1], -t[i][1]]
    
    # Execute singular value decomposition
    u, s, vh = np.linalg.svd(A)
    
    # Minimizing by taking last column of V, generating homography
    homography = np.ones((3,3))
    count = 0
    for i in range(3):
        for j in range(3):
            homography[i,j] = vh[8,count]
            count += 1
    return homography


In [133]:
def rescan_image(image, cv2_detections, threshold=0.1):
    """
    Rescan an image for QR codes based on approximate locations from initial detections.
    
    Args:
        image (ndarray): The input image.
        cv2_detections (list): List of initial detections with bounding box coordinates.
        threshold (float): Threshold to determine if a QR code is within the same area.
            A lower value means stricter matching.
    
    Returns:
        rescanned_detections (list): List of valid detections that were successfully rescanned.
    """
    rescanned_detections = []
    qr_detector = cv2.QRCodeDetector()

    for detection in cv2_detections:
        # Extract and pad the bounding box
        x_min = detection['x_min']
        y_min = detection['y_min']
        x_max = detection['x_max']
        y_max = detection['y_max']

        side_length = max(x_max - x_min, y_max - y_min)
        pad = side_length * 3

        x_min_padded = max(0, int(x_min - pad))
        y_min_padded = max(0, int(y_min - pad))
        x_max_padded = min(image.shape[1], int(x_max + pad))
        y_max_padded = min(image.shape[0], int(y_max + pad))

        # Extract the region of interest
        roi = image[y_min_padded:y_max_padded, x_min_padded:x_max_padded]

        # Rescan the region for QR codes
        data, _, vertices, _ = qr_detector.detectAndDecodeMulti(roi)

        if vertices is not None:
            for i, points in enumerate(vertices):
                # Convert local coordinates back to global
                points_global = points + [x_min_padded, y_min_padded]
                x_min_global = points_global[:, 0].min()
                y_min_global = points_global[:, 1].min()
                x_max_global = points_global[:, 0].max()
                y_max_global = points_global[:, 1].max()

                # Calculate overlap with the original detection
                overlap_x = max(0, min(x_max, x_max_global) - max(x_min, x_min_global))
                overlap_y = max(0, min(y_max, y_max_global) - max(y_min, y_min_global))
                overlap_area = overlap_x * overlap_y
                original_area = (x_max - x_min) * (y_max - y_min)

                # Check if the rescan is close enough to the original detection
                if overlap_area / original_area >= threshold:
                    rescanned_detections.append({
                        'x_min': x_min_global,
                        'y_min': y_min_global,
                        'x_max': x_max_global,
                        'y_max': y_max_global,
                        'label': f"QR {i}"
                    })

    return rescanned_detections

def validate_and_redetect(image, detections, template, pyramid_levels=3, threshold=0.0):
    """
    Validate initial detections and attempt redetection if validation fails.

    Args:
        image (ndarray): The input image.
        detections (list): List of initial detections with bounding box coordinates.
        template (ndarray): Template of the QR code to search for.
        pyramid_levels (int): Number of pyramid levels for validation and redetection.
        threshold (float): Minimum confidence for template matching.

    Returns:
        updated_detections (list): List of validated or redetected bounding boxes.
    """
    updated_detections = []
    h, w = image.shape[:2]

    # Create Laplacian pyramid for the image
    image_pyramid = [image]
    for _ in range(pyramid_levels - 1):
        image_pyramid.append(cv2.pyrDown(image_pyramid[-1]))

    for detection in detections:
        # Extract ROI around the detection
        x_min, y_min, x_max, y_max = (
            int(detection['x_min']),
            int(detection['y_min']),
            int(detection['x_max']),
            int(detection['y_max']),
        )

        roi = image[y_min:y_max, x_min:x_max]
        resized_template = cv2.resize(template, (roi.shape[1], roi.shape[0]))

        # Validate using template matching within the ROI
        result = cv2.matchTemplate(roi, resized_template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        print("match result: ",cv2.minMaxLoc(result))

        if max_val >= threshold:
            # If validation succeeds, adjust coordinates to the original scale
            updated_detections.append({
                'x_min': x_min,
                'y_min': y_min,
                'x_max': x_max,
                'y_max': y_max,
                'confidence': max_val
            })

    return updated_detections

In [ ]:
filepath = "../data/test/samples/test_marks.JPG"
# Load the model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../models/yolov5n_best.pt')

image_name = os.path.splitext(os.path.basename(filepath))[0]
print(f"Processing: {filepath}")
image = cv2.imread(filepath)
if image is None:
    raise ValueError(f"Error: Could not load image at {filepath}")


result, stats = measure_processing_power(qr_code_detector, image)
cv2_detections, cv2_labels = result
print(cv2_detections)
print(stats)
save_annotated_image( image, cv2_detections, "../data/targets/sandbox_run/cv2", image_name)

#print(rescan_image(image, cv2_detections, threshold=0.1))
template = cv2.imread("../data/qr_codes/qr_tr.png")
result, stats = measure_processing_power(validate_and_redetect, image, cv2_detections, template, pyramid_levels=3, threshold=0.0)
print(stats)
filtered_detections = result.copy()
save_annotated_image(image, filtered_detections, "../data/tests/sandbox_run/cv2", image_name, suffix="_tmatch")

src_points, tl_point, br_point = format_detections_for_transform(filtered_detections, image.shape)

Using cache found in /Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-12 Python-3.11.11 torch-2.5.1 CPU

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 


Processing: ../data/test/samples/test_marks.JPG
[{'x_min': 3317.0, 'y_min': 2600.667, 'x_max': 3557.0, 'y_max': 2894.2947, 'label': 'QR 0'}, {'x_min': 3360.4148, 'y_min': 1623.0, 'x_max': 3615.5322, 'y_max': 1899.5979, 'label': 'QR 1'}, {'x_min': 2364.7778, 'y_min': 1623.0138, 'x_max': 2547.1968, 'y_max': 1855.8821, 'label': 'QR 2'}, {'x_min': 1993.0, 'y_min': 21.0, 'x_max': 5709.0, 'y_max': 1544.0, 'label': 'QR 3'}, {'x_min': 2328.4646, 'y_min': 2476.3237, 'x_max': 2507.452, 'y_max': 2722.0, 'label': 'QR 4'}]
{'CPU_Usage (%)': 116.3, 'Memory_Usage (bytes)': 208715776, 'Duration (seconds)': 2.850655124988407}
Saved annotated image: ../data/targets/scanned/cv2/images/test_marks_result.jpg
match result:  (0.09911054372787476, 0.09911054372787476, (0, 0), (0, 0))
match result:  (0.10503210872411728, 0.10503210872411728, (0, 0), (0, 0))
match result:  (0.013012969866394997, 0.013012969866394997, (0, 0), (0, 0))
match result:  (-0.14484745264053345, -0.14484745264053345, (0, 0), (0, 0))
mat

In [135]:
homography = find_four_point_transform(src_points, dst_points)

In [149]:
def correspondences(x,y,eig):
    
    num_x = eig[0][0] * x + eig[0][1] * y + eig[0][2]
    den_x = eig[2][0] * x + eig[2][1] * y + eig[2][2]
    
    num_y = eig[1][0] * x + eig[1][1] * y + eig[1][2]
    den_y = eig[2][0] * x + eig[2][1] * y + eig[2][2]
    
    x_t = num_x / den_x
    y_t = num_y / den_y
    
    return x_t, y_t

def crop_to_corners(image, tl_point, br_point, eig):


    # transform corners
    x_tl, y_tl = correspondences(tl_point[0], tl_point[1], eig)
    x_br, y_br = correspondences(br_point[0], br_point[1], eig)
    x_tl, y_tl = int(x_tl), int(y_tl)
    x_br, y_br = int(x_br), int(y_br)

    # Crop the image
    cropped_image = image[y_tl:y_br, x_tl:x_br]

    return cropped_image


def crop_to_qr_corners(image, detections, eig):
    """
    Crops the image from the top-left QR code corner to the bottom-right QR code corner.

    Args:
        image (ndarray): The input image to be cropped.
        detections (list): List of dictionaries with bounding box details for QR codes.
        eig (ndarray): Homography matrix.

    Returns:
        ndarray: Cropped image.
    """
    # Ensure there are at least two QR codes detected
    if len(detections) < 2:
        raise ValueError("At least two QR codes (top-left and bottom-right) are required to crop the image.")

    # Find the top-left and bottom-right QR codes
    src_points = format_detections_for_transform(detections, image.shape)
    top_left = src_points[0].astype(int)  # Top-left QR code
    bottom_right = src_points[2].astype(int)  # Bottom-right QR code

    # Transform the top-left and bottom-right points using the homography matrix
    x_tl, y_tl = correspondences(top_left[0], top_left[1], eig)
    x_br, y_br = correspondences(bottom_right[0], bottom_right[1], eig)

    # Ensure the points are integers for cropping
    pad = 200
    x_tl, y_tl = int(x_tl), int(y_tl)
    x_br, y_br = int(x_br), int(y_br)

    # Crop the image
    cropped_image = image[y_tl:y_br, x_tl:x_br]

    return cropped_image

In [150]:
warped = cv2.warpPerspective(image, homography, (image.shape[1],image.shape[0]))
warped_and_cropped = crop_to_corners(warped, tl_point, br_point, homography)

In [ ]:
output_size = (110,85)
cv2.imwrite("../data/tests/sandbox_run/cv2/images/output_warped_cropped.jpg",warped_and_cropped)

True